<a href="https://colab.research.google.com/github/gtbnhyujmj/Auto_Shorts_Maker/blob/main/movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# 安裝所需套件
# !pip install moviepy opencv-python-headless --quiet

In [79]:
import cv2
import os
import numpy as np
from moviepy.editor import VideoFileClip, ImageSequenceClip
from PIL import Image

# 設定資料夾路徑

In [80]:
# In Google Drive

bg_folder = "/content/drive/MyDrive/shorts/backgrounds_MP4"
char_folder = "/content/drive/MyDrive/shorts/characters"
overlay_folder = "/content/drive/MyDrive/shorts/overlays"

# 載入人物與上層圖案

In [81]:
char_img = cv2.imread(os.path.join(char_folder, "char1.png"), cv2.IMREAD_UNCHANGED)
overlay_img = cv2.imread(os.path.join(overlay_folder, "top_icon.png"), cv2.IMREAD_UNCHANGED)

# 工具函數：將 PNG 貼到背景圖（支援透明度）

In [82]:
def overlay_image(background, overlay, position):
    x, y = position
    bh, bw = background.shape[:2]
    oh, ow = overlay.shape[:2]

    # 範圍裁切處理
    if x < 0:
        overlay = overlay[:, -x:]
        ow = overlay.shape[1]
        x = 0
    if y < 0:
        overlay = overlay[-y:, :]
        oh = overlay.shape[0]
        y = 0
    if x + ow > bw:
        ow = bw - x
        overlay = overlay[:, :ow]
    if y + oh > bh:
        oh = bh - y
        overlay = overlay[:oh, :]

    if overlay.shape[2] == 4:
        alpha = overlay[:, :, 3] / 255.0
        alpha = np.stack([alpha]*3, axis=-1)
        foreground = overlay[:, :, :3].astype(np.float32)
        background_crop = background[y:y+oh, x:x+ow].astype(np.float32)
        blended = alpha * foreground + (1 - alpha) * background_crop
        background[y:y+oh, x:x+ow] = blended.astype(np.uint8)
    else:
        background[y:y+oh, x:x+ow] = overlay[:, :, :3]

    return background

# 定義合成函數

In [83]:
def compose_frame(bg_frame, char_img, overlay_img, char_offset_ratio):
    h, w = bg_frame.shape[:2]  # 影片或背景的大小，應該是 1920 高

    output = bg_frame.copy()

    # === ⬇️ 處理人物圖像（高度變成和背景一樣高） ===
    char_scale_ratio = float(h) / char_img.shape[0]
    new_char_width = int(char_img.shape[1] * char_scale_ratio)
    char_resized = cv2.resize(char_img, (new_char_width, h))  # 高度 = 1920

    # 人物 x, y 位置（中間 + offset）
    char_x = (w - new_char_width) // 2
    char_y = int(h * char_offset_ratio)  # 可為正（往下）或負（往上）

    output = overlay_image(output, char_resized, (char_x, char_y))

    # === ⬇️ 處理上方 overlay 圖像（高度 = 1920） ===
    overlay_scale_ratio = float(h) / overlay_img.shape[0]
    new_overlay_width = int(overlay_img.shape[1] * overlay_scale_ratio)
    overlay_resized = cv2.resize(overlay_img, (new_overlay_width, h))

    # overlay 固定放中軸下方約 33%
    overlay_x = (w - new_overlay_width) // 2
    overlay_y = int(h * 0.75)

    output = overlay_image(output, overlay_resized, (overlay_x, overlay_y))

    return output

# 合成主程式

In [84]:
bg_file = os.path.join(bg_folder, "bg1.mp4")  # 或 bg1.png
is_video = bg_file.endswith(".mp4")

output_path = "/content/output.mp4" if is_video else "/content/output.png"

if is_video:
    clip = VideoFileClip(bg_file)
    output_frames = []
    for frame in clip.iter_frames():
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        composed = compose_frame(frame_bgr, char_img, overlay_img, char_offset_ratio=0.0)
        output_frames.append(cv2.cvtColor(composed, cv2.COLOR_BGR2RGB))
    out_clip = ImageSequenceClip(output_frames, fps=clip.fps)
    out_clip.write_videofile(output_path, codec="libx264")
else:
    bg_img = cv2.imread(bg_file)
    composed = compose_frame(bg_img, char_img, overlay_img, char_offset_y=0.0)
    cv2.imwrite(output_path, composed)

print(f"合成完成！輸出檔案：{output_path}")

Moviepy - Building video /content/output.mp4.
Moviepy - Writing video /content/output.mp4



Moviepy - Done !
Moviepy - video ready /content/output.mp4
合成完成！輸出檔案：/content/output.mp4


# 下次要建立啥?

In [86]:
# 建立素材在資料夾裡面的 "隨機 shaffule"
# 圖片高度可能不必1920，人物的圖案有點太大。

In [87]:
# STU, 2025/05/21